<a href="https://colab.research.google.com/github/HimalKarkal/netball-analysis/blob/master/Glicko_Rating_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# Cloning the data from GitHub

! git clone 'https://github.com/HimalKarkal/netball-analysis.git'

Cloning into 'netball-analysis'...
remote: Enumerating objects: 26871, done.
remote: Counting objects: 100% (3099/3099), done.
remote: Compressing objects: 100% (2129/2129), done.
remote: Total 26871 (delta 881), reused 3076 (delta 870), pack-reused 23772
Receiving objects: 100% (26871/26871), 45.30 MiB | 19.79 MiB/s, done.
Resolving deltas: 100% (13068/13068), done.
Updating files: 100% (14890/14890), done.


# Defining the Glicko Rating System

In [2]:
import math

class GlickoRating:
    def __init__(self, r=1500, rd=350):
        self.r = r
        self.rd = rd

q = math.log(10) / 400

def g(rd):
  return 1 / math.sqrt(1 + (3 * q**2 * rd**2) / math.pi**2)

def E(r, ro, rdo):
    return 1 / (1 + 10**(-g(rdo) * (r - ro) / 400))

def o(margin, positiveStat = True):
  if positiveStat == True:
    return 1 / (1 + math.exp(-margin))
  else:
    return 1 / (1 + math.exp(margin))

def d2(r, ro, rdo):
  return (q**2 * g(rdo)**2 * E(r, ro, rdo) * (1 - E(r, ro, rdo)) + 1e-9)**-1 #Added epsilon to prevent error.

def rate(ratingP, ratingO, margin, positiveStat=True):
  r = ratingP.r
  rd = ratingP.rd
  ro = ratingO.r
  rdo = ratingO.rd
  e = E(r, ro, rdo)

  if positiveStat == True:
    outcome = o(margin, positiveStat = True)
  else:
    outcome = o(margin, positiveStat = False)

  r_updated = r + (q / ((1 / rd**2) + 1 / d2(r, ro, rdo))) * g(rdo) * (outcome - e)
  rd_updated = math.sqrt(1 / ((1 / rd**2) + 1 / d2(r, ro, rdo)))

  return GlickoRating(r_updated, rd_updated)

#Defining Constants and Helper Functions

In [10]:
exp = {
      'goal1':{},
      'goal2':{},
      'rebounds':{},
      'feedWithAttempt':{},
      'feedWithoutAttempt':{},
      'pickups':{},
      'centrePassReceives':{},
      'gain':{},
      'deflections':{},
      'blocks':{},
      'blocked':{},
      'goalMisses':{},
      'generalPlayTurnovers':{},
      'penalties':{}
      }

In [4]:
#DEFINING CONSTANTS

import pandas as pd
import numpy as np
import glob

# Constants
TOURNAMENT = "SSN"
SEASONS = ['2020', '2021', '2022', '2023']
DICT_WEIGHTS = {
'GS':{
    "goal1": 0.35,
    "goalMisses": 0.2,
    "goal2": 0.15,
    "generalPlayTurnovers": 0.1,
    "rebounds": 0.07,
    "blocked": 0.05,
    "penalties": 0.05,
    "feedWithAttempt": 0.01,
    "feedWithoutAttempt": 0.01,
    "pickups": 0.01
},

'GA':{
    "goal1": 0.25,
    "goalMisses": 0.15,
    "goal2": 0.1,
    "feedWithAttempt": 0.1,
    "generalPlayTurnovers": 0.1,
    "centrePassReceives": 0.07,
    "feedWithoutAttempt": 0.07,
    "blocked": 0.05,
    "rebounds": 0.05,
    "penalties": 0.04,
    "gain": 0.01,
    "pickups": 0.01
},

'WA':{
    "centrePassReceives": 0.3,
    "feedWithAttempt": 0.25,
    "generalPlayTurnovers": 0.15,
    "feedWithoutAttempt": 0.1,
    "penalties": 0.07,
    "gain": 0.05,
    "pickups": 0.05,
    "deflections": 0.03
},

'C': {
    "generalPlayTurnovers": 0.35,
    "feedWithAttempt": 0.3,
    "feedWithoutAttempt": 0.15,
    "penalties": 0.07,
    "pickups": 0.07,
    "gain": 0.05,
    "deflections": 0.01
},

'WD': {
    "deflections": 0.3,
    "penalties": 0.25,
    "gain": 0.2,
    "generalPlayTurnovers": 0.1,
    "centrePassReceives": 0.07,
    "pickups": 0.05,
    "feedWithAttempt": 0.02,
    "feedWithoutAttempt": 0.01
},

'GD': {
    "deflections": 0.35,
    "penalties": 0.2,
    "gain": 0.15,
    "blocks": 0.1,
    "rebounds": 0.1,
    "generalPlayTurnovers": 0.05,
    "centrePassReceives": 0.03,
    "pickups": 0.02
},

'GK': {
    "deflections": 0.35,
    "penalties": 0.25,
    "gain": 0.15,
    "blocks": 0.1,
    "rebounds": 0.1,
    "generalPlayTurnovers": 0.03,
    "pickups": 0.02
}
}

In [3]:
# DEFINING LOAD FILES FUNCTION

def load_files(tournament, season, file_type):
    file_paths = glob.glob(f"/content/netball-analysis/data/matchCentre/processed/*/*{file_type}*{season}*{tournament}*.csv")
    file_paths.sort()
    return file_paths

In [5]:
# CLASSIFYING PLAYERS FUNCTION

def classify_player_positions(df_subs):
    position_dict = {'GS': {}, 'GA': {}, 'WA': {}, 'C': {}, 'WD': {}, 'GD': {}, 'GK': {}, 'S': {}}
    for _, row in df_subs.iterrows():
        player, position, duration = row['playerId'], row['startingPos'], row['duration']
        if player in position_dict[position]:
            position_dict[position][player] += duration
        else:
            position_dict[position][player] = duration
    df_time_in_position = pd.DataFrame(position_dict).fillna(0).reset_index().rename(columns={'index': 'playerId'})
    df_time_in_position = df_time_in_position[df_time_in_position['S'] != 3600]
    df_time_in_position['position'] = df_time_in_position.drop(columns=['playerId', 'S']).idxmax(axis=1)
    return df_time_in_position

In [13]:
#RATING PLAYERS FUNCTION

def calculate_player_ratings(df_player_stats, df_time_in_position, dict_weights, dict_player_ratings, dict_relative_league_ratings, dict_player_stats, dict_league_stats, dict_absolute_ratings):
    df_player_stats = df_player_stats.merge(df_time_in_position[['playerId', 'position']], on='playerId', how='left').dropna()
    df_player_stats['feedWithoutAttempt'] = df_player_stats['feeds'] - df_player_stats['feedWithAttempt']
    relevant_stats = ['goal1', 'goal2', 'rebounds', 'feedWithAttempt', 'feedWithoutAttempt', 'pickups', 'centrePassReceives', 'gain', 'deflections', 'blocks', 'blocked', 'goalMisses', 'generalPlayTurnovers', 'penalties']
    for stat in relevant_stats:
        df_player_stats[stat] = df_player_stats[stat] / df_player_stats['minutesPlayed']
    df_player_stats = df_player_stats[df_player_stats['minutesPlayed'] >= 10]
    df_means, df_stds = df_player_stats.groupby('position').mean().round(2).reset_index(), df_player_stats.groupby('position').std().round(2).reset_index()

    for i, player in df_player_stats.iterrows():
        position, player_id = player['position'], player['playerId']
        row_means, row_stds = df_means[df_means['position'] == position], df_stds[df_stds['position'] == position]

        for stat in dict_weights[position]:
            if stat not in dict_player_ratings:
                dict_player_ratings[stat] = {}
                dict_relative_league_ratings[stat] = {}
                dict_player_stats[stat] = {}
                dict_league_stats[stat] = {}
            if player_id not in dict_player_ratings[stat]:
                dict_player_ratings[stat][player_id] = [GlickoRating()]
                dict_relative_league_ratings[stat][player_id] = [GlickoRating()]
                dict_player_stats[stat][player_id] = []
                dict_league_stats[stat][player_id] = []

            player_rating = dict_player_ratings[stat][player_id][-1]
            league_average_rating = dict_relative_league_ratings[stat][player_id][-1]
            sd = row_stds.iloc[0][stat]
            margin = (player[stat] - row_means.iloc[0][stat]) / sd if sd != 0 else 0
            player_rating = rate(player_rating, league_average_rating, margin, positiveStat=(stat not in ['blocked', 'goalMisses', 'generalPlayTurnovers', 'penalties']))
            league_average_rating = rate(league_average_rating, player_rating, 0, positiveStat=(stat not in ['blocked', 'goalMisses', 'generalPlayTurnovers', 'penalties']))
            dict_player_ratings[stat][player_id].append(player_rating)
            dict_relative_league_ratings[stat][player_id].append(league_average_rating)
            dict_player_stats[stat][player_id].append(player[stat])
            dict_league_stats[stat][player_id].append(row_means.iloc[0][stat])

        if player_id not in dict_absolute_ratings:
            dict_absolute_ratings[player_id] = [1500]
        absolute_rating = sum(dict_weights[position][stat] * dict_player_ratings[stat][player_id][-1].r for stat in dict_weights[position])
        dict_absolute_ratings[player_id].append(absolute_rating)
        df_player_stats.at[i, 'glickoRating'] = absolute_rating

    for player in dict_absolute_ratings:
        if player not in df_player_stats['playerId'].values:
            dict_absolute_ratings[player].append(np.nan)

    max_length = max(len(r) for r in dict_absolute_ratings.values())
    for player_id, ratings in dict_absolute_ratings.items():
        dict_absolute_ratings[player_id] = [np.nan] * (max_length - len(ratings)) + ratings

    return df_player_stats


In [14]:
#PROCESSING SEASONS FOR TEAM RATINGS

def process_season(season, tournament, dict_weights, dict_player_ratings, dict_relative_league_ratings, dict_player_stats, dict_league_stats, dict_absolute_ratings):
    list_subs = load_files(tournament, season, 'substitutions')
    list_player_stats = load_files(tournament, season, 'playerStats')
    list_team_stats = load_files(tournament, season, 'teamStats')

    df_team_ratings = pd.DataFrame()

    for week in range(1, 15):
        df_subs = pd.concat([pd.read_csv(list_subs[4 * (week - 1) + game]) for game in range(4)])
        df_player_stats = pd.concat([pd.read_csv(list_player_stats[4 * (week - 1) + game]) for game in range(4)])
        df_team_stats = pd.concat([pd.read_csv(list_team_stats[4 * (week - 1) + game]) for game in range(4)])

        df_time_in_position = classify_player_positions(df_subs)
        df_player_stats = calculate_player_ratings(df_player_stats, df_time_in_position, dict_weights, dict_player_ratings, dict_relative_league_ratings, dict_player_stats, dict_league_stats, dict_absolute_ratings)

        team_average_ratings = df_player_stats[['squadId', 'glickoRating']].groupby('squadId').mean().round(2).reset_index()
        df_team_stats = df_team_stats.merge(team_average_ratings, on='squadId', how='left')
        df_team_ratings = pd.concat([df_team_ratings, df_team_stats])

    return df_team_ratings

In [26]:
# THE FUNCTION OF FUNCTIONS

def main():
    dict_player_ratings = {stat: {} for stat in exp}
    dict_relative_league_ratings = {stat: {} for stat in exp}
    dict_player_stats = {stat: {} for stat in exp}
    dict_league_stats = {stat: {} for stat in exp}
    dict_absolute_ratings = {}
    df_team_ratings = pd.DataFrame()

    for season in SEASONS:
        list_subs = load_files(TOURNAMENT, season, 'substitutions')
        list_player_stats = load_files(TOURNAMENT, season, 'playerStats')
        list_team_stats = load_files(TOURNAMENT, season, 'teamStats')

        for week in range(1, 15):
            df_subs = pd.concat([pd.read_csv(list_subs[4 * (week - 1) + game]) for game in range(4)])
            df_player_stats = pd.concat([pd.read_csv(list_player_stats[4 * (week - 1) + game]) for game in range(4)])
            df_team_stats = pd.concat([pd.read_csv(list_team_stats[4 * (week - 1) + game]) for game in range(4)])

            df_time_in_position = classify_player_positions(df_subs)
            df_player_stats = calculate_player_ratings(df_player_stats, df_time_in_position, DICT_WEIGHTS, dict_player_ratings, dict_relative_league_ratings, dict_player_stats, dict_league_stats, dict_absolute_ratings)

            team_average_ratings = df_player_stats[['squadId', 'glickoRating']].groupby('squadId').mean().round(2).reset_index()
            df_team_stats = df_team_stats.merge(team_average_ratings, on='squadId', how='left')
            df_team_ratings = pd.concat([df_team_ratings, df_team_stats])

    # Save team ratings to a CSV file
    df_team_ratings.to_csv("team_ratings.csv", index=False)
    print("Team Ratings saved to team_ratings.csv")

    # Convert dict_player_ratings to a DataFrame
    player_ratings_list = []
    for stat, players in dict_player_ratings.items():
        for player, ratings in players.items():
            for rating in ratings:
                player_ratings_list.append([stat, player, rating.r, rating.rd])
    df_player_ratings = pd.DataFrame(player_ratings_list, columns=["stat", "playerId", "rating", "rd"])

    # Save dict_player_ratings to a CSV file
    df_player_ratings.to_csv("player_ratings.csv", index=False)
    print("Player Ratings saved to player_ratings.csv")

    # Convert dict_absolute_ratings to a DataFrame
    absolute_ratings_list = []
    for player, ratings in dict_absolute_ratings.items():
        for week, rating in enumerate(ratings):
            absolute_ratings_list.append([player, week, rating])
    df_absolute_ratings = pd.DataFrame(absolute_ratings_list, columns=["playerId", "week", "absolute_rating"])

    # Save dict_absolute_ratings to a CSV file
    df_absolute_ratings.to_csv("absolute_ratings.csv", index=False)
    print("Absolute Ratings saved to absolute_ratings.csv")

if __name__ == "__main__":
    main()

<ipython-input-5-40f9ab1c0c2b>:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_time_in_position['position'] = df_time_in_position.drop(columns=['playerId', 'S']).idxmax(axis=1)


Team Ratings saved to team_ratings.csv
Player Ratings saved to player_ratings.csv
Absolute Ratings saved to absolute_ratings.csv
